In [1]:
reset -f

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
from ipyleaflet import Map, GeoData, basemaps, LayersControl, Marker, Popup
#import json
import ipywidgets as ipyw

In [3]:
gdf = gpd.read_file('data/cb_2019_06_tract_500k/cb_2019_06_tract_500k.shp')

In [4]:
# get screwed (replace with real metrics later)
screwed_dummy = gdf.sample(740)

In [5]:
def add_screwed(row):
    if row['GEOID'] in list(screwed_dummy['GEOID']):
        val = 1
    else:
        val = 0
    return(val)

"""
def add_color(row):
    if row['screwed'] == 1:
        val = '#34eb4f'
    else:
        val = '#eb7a34'
    return(val)
"""
print()

In [6]:
gdf['screwed'] = gdf.apply(add_screwed, axis=1)
# gdf['color'] = gdf.apply(add_color, axis=1)

In [7]:
# split dataframes into nonscrewed and screwed
gdf_screwed = gdf[gdf['screwed'] == 1]
gdf_therest = gdf[gdf['screwed'] == 0]
# print((len(gdf)) == (len(gdf_therest) + (len(gdf_screwed))))

In [8]:
# prepping map
zoom = 6
center = [37.248, -119.432]

In [9]:
# prepping layers for map
ct_therest = GeoData(geo_dataframe = gdf_therest,
                  style={'color': 'black', 'fillColor': '#34eb4f', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                  hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                  name = 'GEOID'
                 )
ct_screwed = GeoData(geo_dataframe = gdf_screwed,
                  style={'color': 'black', 'fillColor': '#eb7a34', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                  hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                  name = 'GEOID'
                 )

In [10]:
"""
message = ipyw.HTML()
message.value = "braindead"
message.description = "html?"
"""

'\nmessage = ipyw.HTML()\nmessage.value = "braindead"\nmessage.description = "html?"\n'

In [11]:
# dir(ct_therest.popup)

In [12]:
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

# add the layers
m.add_layer(ct_therest)
m.add_layer(ct_screwed)

#ct_screwed.popup = message

## Note: 'screwed' census tracts are selected randomly here for testing purposes.

In [13]:
display(m)

Map(center=[37.248, -119.432], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…